In [9]:
import os

def load_zshrc_env(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('export'):
                key_value = line.strip().split(' ')[1].split('=')
                key = key_value[0]
                value = key_value[1].strip('"').strip("'")
                os.environ[key] = value

zshrc_path = os.path.expanduser('~/.zshrc')
load_zshrc_env(zshrc_path)

# Now you can access your zshrc environment variables using os.environ
print(os.environ)


environ({'COHERE_API_KEY': 'KlH7U30p7fdagLGOgHzacD4YKZwRQPdt0H2YKMEW', 'COLORTERM': 'truecolor', 'COMMAND_MODE': 'unix2003', 'CONDA_CHANGEPS1': 'false', 'DEEPGRAM_API_KEY': 'ce3b0af005fc7e49a4c4d3615c49f47f5dbc358f', 'ELECTRON_NO_ATTACH_CONSOLE': '1', 'FIG_PID': '68900', 'FIG_SET_PARENT_CHECK': '1', 'HOME': '/Users/qinliao', 'HOMEBREW_CELLAR': '/opt/homebrew/Cellar";', 'HOMEBREW_PREFIX': '/opt/homebrew";', 'HOMEBREW_REPOSITORY': '/opt/homebrew";', 'INFOPATH': '/opt/homebrew/share/info:${INFOPATH:-}";', 'LANG': 'en_US.UTF-8', 'LESS': '-R', 'LOGNAME': 'qinliao', 'LSCOLORS': 'Gxfxcxdxbxegedabagacad', 'LS_COLORS': 'di=1;36:ln=35:so=32:pi=33:ex=31:bd=34;46:cd=34;43:su=30;41:sg=30;46:tw=30;42:ow=30;43', 'LaunchInstanceID': '9986FC31-9E72-4308-B2D9-AF91D646F227', 'MANPATH': '/opt/homebrew/share/man${MANPATH+:$MANPATH}:";', 'MallocNanoZone': '0', 'NVM_BIN': '/Users/qinliao/.nvm/versions/node/v20.0.0/bin', 'NVM_CD_FLAGS': '-q', 'NVM_DIR': '$HOME/.nvm', 'NVM_INC': '/Users/qinliao/.nvm/versions/n

In [10]:
from langchain.document_loaders import YoutubeLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.schema import Document
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [11]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

directories = [
    '/Users/qinliao/learn/knausj_talon',
    '/Users/qinliao/learn/wiki',    
    '/Users/qinliao/.talon/user/cursorless-talon',
    '/Users/qinliao/.talon/user/cursorless-settings'
]

docs = []

for directory in directories:
    for dirpath, dirnames, filenames in os.walk(directory):
        for file in filenames:
            try:
                loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
                docs.extend(loader.load_and_split())
            except Exception as e:
                pass

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs)


Created a chunk of size 1384, which is longer than the specified 1000
Created a chunk of size 1251, which is longer than the specified 1000
Created a chunk of size 1091, which is longer than the specified 1000
Created a chunk of size 1178, which is longer than the specified 1000
Created a chunk of size 1040, which is longer than the specified 1000
Created a chunk of size 1090, which is longer than the specified 1000
Created a chunk of size 2910, which is longer than the specified 1000
Created a chunk of size 1840, which is longer than the specified 1000
Created a chunk of size 2110, which is longer than the specified 1000
Created a chunk of size 1559, which is longer than the specified 1000
Created a chunk of size 1295, which is longer than the specified 1000
Created a chunk of size 1077, which is longer than the specified 1000
Created a chunk of size 1366, which is longer than the specified 1000
Created a chunk of size 1098, which is longer than the specified 1000
Created a chunk of s

In [12]:
embeddings = CohereEmbeddings()
vectorstore = Chroma.from_documents(
    docs, embeddings, persist_directory='db', verbose=True
)

Using embedded DuckDB with persistence: data will be stored in: db


: 

: 

In [ ]:
query = "uppercase in talon?"
docs = vectorstore.similarity_search_with_score(query)
print(docs[1])

(Document(page_content='from talon import Module', metadata={'source': '/Users/qinliao/.talon/user/cursorless-talon/src/actions/actions_simple.py'}), 6392.228515625)


In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Generally, answer with how the user would do in in talon -- not how to implement it. 
.talon files: spoken work commands that users can use to activate certain commands.
.python files: implementations of the commands that are called by the .talon files.\
.csv files: configurations and settings that users can change


{context}

Question: {question}
Answer: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
prompt_template.format(question=query, context=docs)

'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nGenerally, answer with how the user would do in in talon -- not how to implement it. \n.talon files: spoken work commands that users can use to activate certain commands.\n.python files: implementations of the commands that are called by the .talon files..csv files: configurations and settings that users can change\n\n\n[Document(page_content="## Basic Usage\\n\\nThe examples below are just a very small selection of common commands for working with apps, tabs, media, mouse, etc that should help you be productive with Talon right away.  These are based on the [knaus_talon](https://github.com/knausj85/knausj_talon) Talon user file set (see [the earlier section](#install-a-talon-user-file-set)). These commands may vary depending on your individual setup.\\n\\n\\n### Command history\\n\\n`command history`\\n: Toggles the com

In [ ]:
raw_questions = [
    "how do you switch to using chrome"
    # "how do i tell Talon to use ruby language when editing .rake files",
    # "Is there a way  to get  letters in dictation mode ? For ex for numbers, I can say 'numeral 10' and get 10 vs ten, but  I can't find a way to have it type specific letters in dictation mode. I know how to do so in command mode - thx",
    # "Is there a way to have conformer auto-capitalize names in dictation mode?",
    # "is there were way to control the level of debug information on launch?",
    # "Is there a spoken word command that allows me to hold a key? Example: hold shift while pressing right three times."
]

In [ ]:
prompt_template.format({"question": raw_questions[0]})

KeyError: 'context'

In [ ]:
for question in raw_questions:
    print('#'*20)
    print("question:", question)
    result  = vectorstore_chain({'query': question})
    print(f'answer: {result["result"]}')
    print(f'source: {result["source_documents"]}')
    print('#'*20)


####################
question: how do you switch to using chrome
answer: I'm sorry, I cannot determine the specific context in which you are asking this question. Can you please provide more information or context?
source: [Document(page_content="## Basic Usage\n\nThe examples below are just a very small selection of common commands for working with apps, tabs, media, mouse, etc that should help you be productive with Talon right away.  These are based on the [knaus_talon](https://github.com/knausj85/knausj_talon) Talon user file set (see [the earlier section](#install-a-talon-user-file-set)). These commands may vary depending on your individual setup.\n\n\n### Command history\n\n`command history`\n: Toggles the command history display, which shows the last few commands as Talon understood them; if you're not sure what Talon is doing, or if it understood you correctly, this is how to check it.\n\n\n### Show available comands\n\n`knausj_talon` has an integrated help.  It can show you a 

In [ ]:
vectorstore_chain({'query': "what are all the commands I have to do with mouse? and how can you use them?"})

{'query': 'what are all the commands I have to do with mouse? and how can you use them?',
 'result': ' You can use the `mouse_move()` command to move the mouse to a specific screen coordinate, `mouse_scroll()` to scroll the mouse, `mouse_click()` to click the left or right mouse button, and `mouse_button()` to list the mouse button words to use with the `mouse_click()` index parameter. You can also use the `mouse_wake()` command to enable zoom mouse mode, and the `mouse_hide()` command to hide the mouse cursor.',
 'source_documents': [Document(page_content='```\nimport io;old=sys.stdout;sys.stdout = io.StringIO();actions.list();clip.set_text(sys.stdout.getvalue());sys.stdout = old\n```\n\nSome of the more useful actions are:\n\n* `key(ctrl-a)` - Presses the keys within the parentheses. See the \'Add new keyboard shortcuts\' recipe below for some more info about using this action.\n* `insert("my text")` - Types in the text "my text"\n* `sleep(100ms)` - Waits for 100 milliseconds. This c

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [ ]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = [
    Tool.from_function(
        name = "Search",
        func=vectorstore_chain.__call__, #chain doesn't work anymore because of some inspection issue and there is no call function
        coroutine=vectorstore_chain.acall, #if you want to use async
        description="useful to answer factual questions"
    ),
]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm=ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)


agent_chain.run(input=" how do i tell Talon to use ruby language when editing .rake files")
print(agent_chain.run(input="how do i tell Talon to use ruby language when editing .rake files"))



> Entering new AgentExecutor chain...
{
    "action": "Search",
    "action_input": "How to set Talon to use Ruby language for .rake files"
}
Observation: {'query': 'How to set Talon to use Ruby language for .rake files', 'result': ' Talon does not require special configuration for different programming languages, but many users have per-language customizations to improve efficiency. Checkout the section on Programming Languages in the knausj Talon user file set README for more information on how to use different language modes included in knausj.', 'source_documents': [Document(page_content="# FAQ\n\n### What can I say?  \nCheckout the [common commands](/getting_started/#basic-usage) to get started using Talon.\n\n### How can I code in different languages with Talon\nTalon does not require special configuration for different programming languages, but many users have per-language customizations to improve efficiency. Checkout the section on [Programming Languages](https://github.com

'Talon does not require special configuration for different programming languages, but many users have per-language customizations to improve efficiency. Checkout the section on Programming Languages in the knausj Talon user file set README for more information on how to use different language modes included in knausj.'